In [48]:
# !pip install --upgrade google-api-python-client

In [49]:
# !pip install --upgrade google-auth-oauthlib google-auth-httplib2

In [50]:
import sys
sys.path.append('../') 

from config.youtube import YOUTUBE_API_KEY

from googleapiclient.discovery import build
import pandas as pd
from IPython.display import JSON
import urllib.request
import re

Los proyectos que habilitan la API de datos de YouTube tienen una asignación de cuota predeterminada de 10,000 unidades por día, una cantidad suficiente para la gran mayoría de los usuarios de la API. La cuota predeterminada, que está sujeta a cambios, nos ayuda a optimizar las asignaciones de cuotas y a escalar nuestra infraestructura de una manera que sea más significativa para nuestros usuarios de API. Puedes ver el uso de tu cuota en la página Cuotas en la Consola de API.

Nota: Si alcanzas el límite de cuota, puedes solicitar un aumento del cuota a través del formulario de solicitud de extensión de cuota para los servicios de la API de YouTube.

Calcula el uso de la cuota

Google calcula el uso de tu cuota mediante la asignación de un costo a cada solicitud. Los diferentes tipos de operaciones tienen diferentes costos de cuotas. Por ejemplo:

Una operación de lectura que recupera una lista de recursos (canales, videos o listas de reproducción) suele costar 1 unidad.
Una operación de escritura que crea, actualiza o borra un recurso suele tener un costo de 50 unidades.
Una solicitud de búsqueda cuesta 100 unidades.
La carga de un video cuesta 1600 unidades.

## Definición API

In [51]:
api_key = YOUTUBE_API_KEY
api_service_name = "youtube"
api_version = "v3"

youtube = build(
    api_service_name, api_version, developerKey=api_key)

## Obtencion comentarios en videos del Canal

Buscar en el codigo fuente de la pagina para obtener el ID <br>
https://www.youtube.com/channel/

### Obtención de Id del canal

In [52]:
def extract_channel_ids(urls):
    all_channels = []
    for url in urls:
        with urllib.request.urlopen(url) as response:
            html_content = response.read().decode('utf-8')

        pattern = r'https://www.youtube.com/channel/([^"]+)'
        matches = re.findall(pattern, html_content)

        all_channels.append(matches[0])

    return all_channels

In [53]:
urls = [
    'https://www.youtube.com/@IbaiLlanos',
    #More channels
]
channel_ids = extract_channel_ids(urls)

In [54]:
def get_channel_stats(youtube, channel_ids):
    all_data = []

    request = youtube.channels().list(
        part="snippet,contentDetails,statistics",
        id=','.join(channel_ids)
    )
    response = request.execute()

    for item in response['items']:
        data = {'channelName': item['snippet']['title'],
                'subscribers': item['statistics']['subscriberCount'],
                'views': item['statistics']['viewCount'],
                'totalVideos': item['statistics']['videoCount'],
                'playlistId': item['contentDetails']['relatedPlaylists']['uploads']
                }
        all_data.append(data)

    return (pd.DataFrame(all_data))

In [55]:
channel_stats = get_channel_stats(youtube, channel_ids)
channel_stats

,channelName,subscribers,views,totalVideos,playlistId
0,Ibai,11500000,3427875625,1419,UUaY_-ksFSQtTGk0y1HA_3YQ


### Obtencion videos del canal

In [56]:
def get_video_ids(youtube, playlist_id, limit=None, output_file="../data/yt_video_ids.txt"):
    video_ids = []
    next_page_token = True
    total_results = 0

    while next_page_token is not None and (limit is None or total_results < limit):
        request = youtube.playlistItems().list(
            part="snippet,contentDetails",
            playlistId=playlist_id,
            maxResults=50  # Máximo permitido por la API
        )
        response = request.execute()

        for item in response['items']:
            video_id = item['contentDetails']['videoId']
            video_ids.append(video_id)
            total_results += 1

            # Escribir el ID del video en el archivo de salida
            with open(output_file, "a") as file:
                file.write(video_id + "\n")

            if limit is not None and total_results == limit:
                break

        next_page_token = response.get('nextPageToken')

    return video_ids


In [57]:
video_ids = []
for index, row in channel_stats.iterrows():
    playlist_id = row['playlistId']
    video_ids.extend(get_video_ids(youtube, playlist_id, limit=1794))

video_ids

['gG_Azt-_48g',
 'oZZMkDZqc1Y',
 '_yMviiQeMgQ',
 'lG6pLRyKxQ0',
 'MaCDv2taRbs',
 '8HsjVIhF8hU',
 'pjZFIW6Krus',
 'LSp9ejLtfes',
 'GqGtGSuPF88',
 'kuNXAJfEm08',
 'DVCJnOSaN0M',
 'Jy_MwmMy3uY',
 '1ij230btR80',
 '_u-V0Jxaqm0',
 'lwrKZ8D5tsg',
 'ycjFhTBIV4U',
 'lqKOKsLnEHo',
 'SQPKYQ0UUt8',
 'zMMHZx9OrTE',
 'FQrDeX6uen8',
 'BDT09AAZYuw',
 'sUp-59uTvtE',
 'AUpW-lRBcn0',
 'oneZ7ziFWeI',
 'c6uMkI-ajdo',
 'v7SW-SXxbFo',
 'OJ3eYTrD0BI',
 '3CS_v2sAlXg',
 'jty92y8TPkk',
 'rNrEnirMKDQ',
 '3zzc2X98uxA',
 'JKeFxt10QNY',
 'DsIhi958ZRs',
 'b659xHxVWgM',
 'IIl1f9QfFqc',
 'mBmUy3KnoXk',
 'vDd8D9MVtEs',
 'WNHt4n-pFS0',
 'pX7CDeUVOno',
 'Jua-I-Ploco',
 'q0FP3hp2cK4',
 'H620z1jozOY',
 'Vp8mjxAYV-8',
 '6i6Lymp_GEc',
 'XK5tNry2mzA',
 '0klq2UXjfKQ',
 'PsdSMjffe-g',
 'Pzrl1x22mQA',
 'KKcXPg-NhFw',
 'MWn--jTEJ8k',
 'gG_Azt-_48g',
 'oZZMkDZqc1Y',
 '_yMviiQeMgQ',
 'lG6pLRyKxQ0',
 'MaCDv2taRbs',
 '8HsjVIhF8hU',
 'pjZFIW6Krus',
 'LSp9ejLtfes',
 'GqGtGSuPF88',
 'kuNXAJfEm08',
 'DVCJnOSaN0M',
 'Jy_MwmMy3uY',
 '1ij230

### Obtencion estadísticas de vídeos

In [58]:
def get_video_details(youtube, video_ids):
        all_video_info = []

        for i in range(0, len(video_ids), 50):
                request = youtube.videos().list(
                        part="snippet,contentDetails,statistics",
                        id=','.join(video_ids[i:i+50])
                )
                response = request.execute()

                for video in response['items']:
                        stats_to_keep = {'snippet': ['channelTitle', 'title', 'description', 'tags', 'publishedAt'],
                                        'statistics': ['viewCount', 'likeCount', 'favouriteCount', 'commentCount'],
                                        'contentDetails': ['duration', 'definition', 'caption']
                                        }
                        video_info = {}
                        video_info['video_id'] = video['id']

                        for k in stats_to_keep.keys():
                                for v in stats_to_keep[k]:
                                        try:
                                                video_info[v] = video[k][v]
                                        except:
                                                video_info[v] = None
                        
                        all_video_info.append(video_info)

        return pd.DataFrame(all_video_info)

In [59]:
#with open('../data/youtube/yt_video_ids.txt', 'r') as file:
    #video_ids = file.read().splitlines()

video_df = get_video_details(youtube, video_ids)
video_df.to_csv('../data/yt_video_stats.csv', sep=';', index=False)
video_df.head(5)

,video_id,channelTitle,title,description,tags,publishedAt,viewCount,likeCount,favouriteCount,commentCount,duration,definition,caption
0,gG_Azt-_48g,Ibai,Me pelo y me la pela,MI CANAL DE DIRECTOS: https://www.youtube.com/...,"[ibai, ibai llanos, reaccionando Ibai, Ibai re...",2024-04-15T13:30:16Z,11554,936,None,16,PT27S,hd,false
1,oZZMkDZqc1Y,Ibai,REACCIONANDO AL SEGUNDO ENTRENO DE PLEX PARA L...,MI CANAL DE DIRECTOS: https://www.youtube.com/...,"[ibai, ibai llanos, reaccionando Ibai, Ibai re...",2024-04-14T19:00:45Z,981533,50132,None,628,PT13M27S,hd,false
2,_yMviiQeMgQ,Ibai,"que os he pillado, venga",MI CANAL DE DIRECTOS: https://www.youtube.com/...,"[ibai, ibai llanos, reaccionando Ibai, Ibai re...",2024-04-14T12:15:03Z,45312,2710,None,35,PT18S,hd,false
3,lG6pLRyKxQ0,Ibai,MI RESPUESTA A DAVID BUSTAMANTE,MI CANAL DE DIRECTOS: https://www.youtube.com/...,"[ibai, ibai llanos, reaccionando Ibai, Ibai re...",2024-04-13T19:45:01Z,695360,37764,None,416,PT8M5S,hd,false
4,MaCDv2taRbs,Ibai,"Si que era cantante, si",MI CANAL DE DIRECTOS: https://www.youtube.com/...,"[ibai, ibai llanos, reaccionando Ibai, Ibai re...",2024-04-13T12:00:16Z,84294,4901,None,63,PT25S,hd,false


### Comentarios

In [60]:
def get_comments_in_videos(youtube, video_ids, limit=None):
    all_comments = []
    
    for video_id in video_ids:
        next_page_token = None  # Inicializamos el token de página para la paginación
        
        while True:
            try:
                request = youtube.commentThreads().list(
                    part="snippet",
                    videoId=video_id,
                    maxResults=100,  # Establecemos un límite máximo por página
                    pageToken=next_page_token
                )
                response = request.execute()
            
                comments_in_video = [
                    {
                        'comment': comment['snippet']['topLevelComment']['snippet']['textOriginal'],
                        'date': comment['snippet']['topLevelComment']['snippet']['publishedAt']
                    }
                    for comment in response.get('items', [])
                ]
                
                all_comments.extend(comments_in_video)
                
                if 'nextPageToken' in response:
                    next_page_token = response['nextPageToken']
                else:
                    break  # No hay más páginas, terminar el bucle de paginación
        
            except Exception as e:
                print(f"Could not get comments for video {video_id}: {str(e)}")
    
    return pd.DataFrame(all_comments)

In [61]:
#with open('../data/youtube/yt_video_ids.txt', 'r') as file:
    #video_ids = file.read().splitlines()

comments_df = get_comments_in_videos(youtube, video_ids , limit=None)
comments_df.to_csv('../data/yt_video_comments.csv', sep=';', index=False)
comments_df

,comment,date
0,Ibai saludas??,2024-04-15T16:45:45Z
1,Hola,2024-04-15T16:42:11Z
2,Jaja vaya mierda,2024-04-15T15:14:07Z
3,"ENDIAMANTADO, SIEMPRE VOLADO🎶",2024-04-15T14:51:49Z
4,Menudo bajon en las colaboraciones de bizarrap...,2024-04-15T14:33:14Z
...,...,...
542375,"Muy Humilde Ibai, Iluminando Todo Su Corazón P...",2024-03-14T21:13:35Z
542376,gracias,2024-03-14T21:13:34Z
542377,.,2024-03-14T21:13:34Z
542378,Primero,2024-03-14T21:13:33Z
